### Part c (Dropout Regularization)

#### Introduction
I implemented a dropout layer in PyTorch using 'nn.dropout'. I found that a lower dropout rate was better, as this is probably because the network is on the smaller size and overfitting is less of a concern. 

#### Results
As you can see below, I achieved a L2-Relative error of 0.045 < 0.05, after using 100,000 epochs, a learning rate of 0.001, and a dropout rate of 0.02.

In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Define the oscillatory function
def f(x):
    result = torch.zeros_like(x)
    result[x < 0] = 5 + sum(torch.sin(k * x[x < 0]) for k in range(1, 5))
    result[x >= 0] = torch.cos(10 * x[x >= 0])
    return result

In [3]:
# Generate training data
x_train = torch.linspace(-np.pi, np.pi, 80)
y_train = f(x_train) + torch.randn(x_train.size()) * 0.1  # Adding Gaussian noise

# Generate testing data
x_test = torch.linspace(-np.pi, np.pi, 1000)
y_test = f(x_test)

In [4]:
# Define the neural network with dropout
class ReLUNetWithDropout(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(ReLUNetWithDropout, self).__init__()
        self.fc1 = nn.Linear(1, 50)  # Input layer to hidden layer with 50 neurons
        self.dropout1 = nn.Dropout(dropout_rate)  # Dropout layer after the first hidden layer
        self.fc2 = nn.Linear(50, 50)  # Hidden layer to another hidden layer with 50 neurons
        self.dropout2 = nn.Dropout(dropout_rate)  # Dropout layer after the second hidden layer
        self.fc3 = nn.Linear(50, 1)  # Hidden layer to output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x.unsqueeze(1)))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x.squeeze()

In [32]:
# Instantiate the model, loss function, and optimizer
dropout_rate = 0.02
model = ReLUNetWithDropout(dropout_rate=dropout_rate)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [33]:
# Train the model
epochs = 100000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(x_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0 or epoch == epochs-1:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 8.863788604736328
Epoch 500, Loss: 0.504927933216095
Epoch 1000, Loss: 0.34009379148483276
Epoch 1500, Loss: 0.27647072076797485
Epoch 2000, Loss: 0.22000804543495178
Epoch 2500, Loss: 0.20072364807128906
Epoch 3000, Loss: 0.23898065090179443
Epoch 3500, Loss: 0.21217553317546844
Epoch 4000, Loss: 0.1772100180387497
Epoch 4500, Loss: 0.16995103657245636
Epoch 5000, Loss: 0.15249080955982208
Epoch 5500, Loss: 0.15438060462474823
Epoch 6000, Loss: 0.1668483167886734
Epoch 6500, Loss: 0.1331923007965088
Epoch 7000, Loss: 0.2237211912870407
Epoch 7500, Loss: 0.1400073766708374
Epoch 8000, Loss: 0.14664529263973236
Epoch 8500, Loss: 0.16639156639575958
Epoch 9000, Loss: 0.1977452039718628
Epoch 9500, Loss: 0.1252157986164093
Epoch 10000, Loss: 0.12161954492330551
Epoch 10500, Loss: 0.20624598860740662
Epoch 11000, Loss: 0.14733660221099854
Epoch 11500, Loss: 0.16684341430664062
Epoch 12000, Loss: 0.12096329778432846
Epoch 12500, Loss: 0.10625214874744415
Epoch 13000, Loss: 0.

In [35]:
# Calculate L2 relative error
model.eval()
with torch.no_grad():
    y_pred_test = model(x_test)
    l2_norm = torch.sqrt(torch.sum((y_pred_test - y_test) ** 2))
    f_norm = torch.sqrt(torch.sum(y_test ** 2))
    l2_relative_error = l2_norm / f_norm
    print(f'L2 Relative Error: {l2_relative_error.item()}')


L2 Relative Error: 0.044689537286758424
